# Tensorflow Music Generator
From [this tutorial](https://www.tensorflow.org/tutorials/audio/music_generation)

In [ ]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import os
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

In [ ]:
seed = 42
# tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [ ]:
data_dir = pathlib.Path.cwd().joinpath("mids")
filenames = glob.glob(str(data_dir.joinpath("*.mid")))

In [ ]:
sample_file = filenames[400]

pm = pretty_midi.PrettyMIDI(sample_file)

In [ ]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
    waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
    
    waveform_short = waveform[:seconds*_SAMPLING_RATE]
    return display.Audio(waveform_short, rate=_SAMPLING_RATE)

# display_audio(pm)

In [68]:
print("Number of instruments: ", len(pm.instruments))
# instrument = pm.instruments[0]
# instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
# print("Instrument name:", instrument_name)
instrument = pm.instruments
for ins in instrument:
    print(pretty_midi.program_to_instrument_name(ins.program))

Number of instruments:  9
Violin
Acoustic Guitar (nylon)
Tenor Sax
String Ensemble 2
Acoustic Grand Piano
Acoustic Bass
Vibraphone
Electric Guitar (jazz)
Whistle


In [ ]:
for i, note in enumerate(instrument.notes[:10]):
    note_name = pretty_midi.note_number_to_name(note.pitch)
    duration = note.end - note.start
    print(f"{i}: pitch={note.pitch}, note_name={note_name},"
          f"duration={duration:.4f}")

In [69]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument_list = []
    for i in range(len(pm.instruments)):
        if pretty_midi.program_to_instrument_name(pm.instruments[i]).lower() in ["guitar", "piano"]:
            instrument_list.append(pm.instruments[i])
    for instrument in instrument_list:
        notes = collections.defaultdict(list)
        
        sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
        prev_start = sorted_notes[0].start
        
        for note in sorted_notes:
            start = note.start
            end = note.end
            notes["pitch"].append(note.pitch)
            notes["start"].append(start)
            notes["end"].append(end)
            notes["step"].append(start - prev_start)
            notes["duration"].append(end - start)
            prev_start = start
        
        return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
raw_notes = midi_to_notes(sample_file)
raw_notes.head()

In [ ]:
get_note_names = np.vectorize(pretty_midi.note_number_to_name)
sample_note_names = get_note_names(raw_notes["pitch"])
sample_note_names[:10]